In [35]:
import os
import logging
from pathlib import Path

import numpy as np
import pandas as pd
import librosa
import torch
import torch.nn.functional as F
from tqdm.auto import tqdm
import sys
from joblib import Parallel, delayed
logging.basicConfig(level=logging.ERROR)


import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [36]:
# 
class CFG:
    def __init__(self, mode="train", kaggle_notebook=False, debug=False):
        assert mode in ["train", "inference"], "mode must be 'train' or 'inference'"
        self.mode = mode
        self.KAGGLE_NOTEBOOK = kaggle_notebook
        self.debug = debug

        # ===== Path Settings =====
        if self.KAGGLE_NOTEBOOK:
            self.OUTPUT_DIR = ''
            self.train_datadir = '/kaggle/input/birdclef-2025/train_audio'
            self.train_csv = '/kaggle/input/birdclef-2025/train.csv'
            self.test_soundscapes = '/kaggle/input/birdclef-2025/test_soundscapes'
            self.submission_csv = '/kaggle/input/birdclef-2025/sample_submission.csv'
            self.taxonomy_csv = '/kaggle/input/birdclef-2025/taxonomy.csv'
            self.spectrogram_npy = '/kaggle/input/birdclef25-mel-spectrograms/birdclef2025_melspec_5sec_256_256.npy'
            
            # kaggle notebookならここを変更
            self.model_path = "/kaggle/input/birdclef-2025-baseline-fold0-0404"
            
            self.device = "cpu"
            self.batch_size = 8
            self.n_jobs = 3
            
        else:
            self.OUTPUT_DIR = '../data/result/'
            self.train_datadir = '../data/raw/train_audio/'
            self.train_csv = '../data/raw/train.csv'
            self.test_soundscapes = '../data/raw/test_soundscapes_small/'
            self.submission_csv = '../data/raw/sample_submission.csv'
            self.taxonomy_csv = '../data/raw/taxonomy.csv'
            self.spectrogram_npy = '../data/processed/mel-spec_0329/birdclef2025_melspec_5sec_256_256.npy'
            self.MODELS_DIR = "../models/"
            
            # ローカルならここを変更
            self.model_path =  "../models/fold0_safezone1000_head_hoplength512_vino/"
            
            self.device = "cuda" if torch.cuda.is_available() else "cpu"
            self.batch_size = 32
            self.n_jobs = 3

        # ===== Model Settings =====
        self.model_name = 'efficientnet_b0'
        self.pretrained = True if mode == "train" else False
        self.in_channels = 1

        # ===== Audio Settings =====
        self.FS = 32000
        self.WINDOW_SIZE = 5
        self.TARGET_DURATION = 5
        self.TARGET_SHAPE = (256, 256)
        self.N_FFT = 1024
        self.HOP_LENGTH = 512
        self.N_MELS = 148
        self.FMIN = 20
        self.FMAX = 16000
        
        self.seed = 42
        
        # smoothingの係数
        self.smooth_center_weight = 0.6
        self.smooth_neighbor_weight = 0.2
        
        # ===== Inference Mode =====
        if mode == "inference":
            self.use_tta = False
            self.tta_count = 3
            self.threshold = 0.5

            self.use_specific_folds = False
            self.folds = [0, 1, 2, 3, 4]  # Used only if use_specific_folds is True

            self.debug_count = 3
            self.ensemble_strategy = "mean" # "mean", "max", "min", "median" など
            
            
            
    def update_debug_settings(self):
        if self.debug:
            self.epochs = 2
            self.selected_folds = [0]

In [37]:
cfg = CFG(mode='inference', kaggle_notebook=False)

if cfg.KAGGLE_NOTEBOOK:
    !pip install -U openvino-telemetry  --no-index --find-links /kaggle/input/pip-hub
    !pip install -U openvino  --no-index --find-links /kaggle/input/pip-hub
    sys.path.append("/kaggle/input/birdclef-2025-libs/")
    
from openvino.runtime import Core
from module import models_lib, utils_lib, preprocess_lib, inference_lib

# Set seed
utils_lib.set_seed(cfg.seed)

In [38]:
print(f"Using device: {cfg.device}")
print(f"Loading taxonomy data...")
taxonomy_df = pd.read_csv(cfg.taxonomy_csv)
species_ids = taxonomy_df['primary_label'].tolist()
num_classes = len(species_ids)
print(f"Number of classes: {num_classes}")

Using device: cuda
Loading taxonomy data...
Number of classes: 206


In [39]:
# mel変換
def process_audio_file(audio_path, cfg):
    """1ファイル分のmelspecデータを返す（row_id, melspecのリスト）"""
    dataset = []
    soundscape_id = Path(audio_path).stem
    try:
        audio_data, _ = librosa.load(audio_path, sr=cfg.FS)
        total_segments = int(len(audio_data) / (cfg.FS * cfg.WINDOW_SIZE))

        for segment_idx in range(total_segments):
            start = int(segment_idx * cfg.FS * cfg.WINDOW_SIZE)
            end = int(start + cfg.FS * cfg.WINDOW_SIZE)
            segment_audio = audio_data[start:end]

            mel_spec = preprocess_lib.process_audio_segment(segment_audio, cfg)
            row_id = f"{soundscape_id}_{(segment_idx + 1) * cfg.WINDOW_SIZE}"

            dataset.append({
                "row_id": row_id,
                "mel_spec": mel_spec
            })
    except Exception as e:
        print(f"Error processing {audio_path}: {e}")
    return dataset


# 並列化してmelspecを生成
def generate_melspec_dataset(cfg):
    test_dir = Path(cfg.test_soundscapes)
    if not test_dir.exists():
        print(f"Test directory {test_dir} does not exist.")
        return []

    test_files = list(test_dir.glob('*.ogg'))
    if len(test_files) == 0:
        print("No test audio files found.")
        return []

    if cfg.debug:
        print(f"Debug mode enabled, using only {cfg.debug_count} files")
        test_files = test_files[:cfg.debug_count]

    results = Parallel(n_jobs=cfg.n_jobs)(
        delayed(process_audio_file)(path, cfg) for path in tqdm(test_files, desc="Parallel melspec gen")
    )
    dataset = [item for sublist in results for item in sublist]
    return dataset

In [40]:

# openvinoモデルの読み込み
def load_openvino_models(vino_dir, cfg):
    models = []
    vino_dir = Path(vino_dir)

    if cfg.use_specific_folds:
        fold_ids = cfg.folds
        xml_files = [vino_dir / f"model_fold{f}.xml" for f in fold_ids]
    else:
        xml_files = sorted(vino_dir.glob("model_fold*.xml"))

    for xml_path in xml_files:
        bin_path = xml_path.with_suffix(".bin")

        if not xml_path.exists() or not bin_path.exists():
            print(f"⚠️ Warning: Missing files for {xml_path.stem}")
            continue

        core = Core()
        model_ir = core.read_model(xml_path)
        compiled_model = core.compile_model(model_ir, device_name="CPU")
        models.append(compiled_model)

        # 🔍 モデルのファイル名（fold情報）をログに出す
        print(f"✅ Loaded model: {xml_path.name}")

    print(f"🎉 Total {len(models)} OpenVINO model(s) loaded from {vino_dir}")
    return models

# openvinoモデルによる推論
def run_inference_openvino(dataset, models_ir, cfg, species_ids):
    row_ids = []
    all_preds = []

    for i in range(0, len(dataset), cfg.batch_size):
        batch = dataset[i:i+cfg.batch_size]

        mel_list = [item["mel_spec"] for item in batch]
        input_tensor = np.stack(mel_list).astype(np.float32)  # (B, H, W)
        input_tensor = np.expand_dims(input_tensor, axis=1)  # (B, 1, H, W)

        preds_per_model = []
        for model in models_ir:
            input_layer = model.input(0)
            output_layer = model.output(0)
            result = model([input_tensor])[output_layer]
            probs = 1 / (1 + np.exp(-result))  # sigmoid
            preds_per_model.append(probs)

        # アンサンブル戦略の選択
        if cfg.ensemble_strategy == "mean":
            avg_preds = np.mean(preds_per_model, axis=0)
        elif cfg.ensemble_strategy == "max":
            avg_preds = np.max(preds_per_model, axis=0)
        elif cfg.ensemble_strategy == "min":
            avg_preds = np.min(preds_per_model, axis=0)
        elif cfg.ensemble_strategy == "median":
            avg_preds = np.median(preds_per_model, axis=0)
        else:
            raise ValueError(f"Unknown ensemble strategy: {cfg.ensemble_strategy}")

        all_preds.append(avg_preds)
        row_ids.extend([item["row_id"] for item in batch])

    predictions = np.concatenate(all_preds, axis=0)
    return row_ids, predictions

In [41]:
def smooth_submission_df(submission_df, cfg, weights=None):
    """
    Smooth predictions using weighted moving average over a 5-frame window: [-2, -1, 0, +1, +2],
    then blend with per-class global average within each soundscape segment group.

    Parameters:
        submission_df: pd.DataFrame with 'row_id' and prediction columns.
        cfg: config object (interface compatibility).
        weights: List of 5 floats (default = [0.1, 0.2, 0.4, 0.2, 0.1]).

    Returns:
        pd.DataFrame with smoothed predictions.
    """
    print("Smoothing submission predictions with global average blend...")

    if weights is None:
        weights = np.array([0.1, 0.2, 0.4, 0.2, 0.1])
    else:
        weights = np.array(weights)

    sub = submission_df.copy()
    cols = sub.columns[1:]
    groups = sub['row_id'].astype(str).str.rsplit('_', n=1).str[0].values
    unique_groups = np.unique(groups)

    for group in unique_groups:
        idx = np.where(groups == group)[0]
        preds = sub.iloc[idx][cols].values  # (T, C)
        T, C = preds.shape

        # エッジ処理：端を繰り返すようにパディング
        padded = np.pad(preds, ((2, 2), (0, 0)), mode='edge')  # (T+4, C)

        # 平滑化：5点加重平均（[-2, -1, 0, +1, +2]）
        smoothed = (
            padded[0:T]   * weights[0] +
            padded[1:T+1] * weights[1] +
            padded[2:T+2] * weights[2] +
            padded[3:T+3] * weights[3] +
            padded[4:T+4] * weights[4]
        )

        # 各クラスの平均予測を20%混ぜる（全セグメントに対して一様に加える）
        classwise_mean = smoothed.mean(axis=0, keepdims=True)  # shape: (1, C)
        smoothed = smoothed * 0.8 + classwise_mean * 0.2

        sub.iloc[idx, 1:] = smoothed

    return sub

In [42]:
print("Generating dataset...")
dataset = generate_melspec_dataset(cfg)

print("Loading OpenVINO models...")
vino_dir = Path(cfg.model_path).with_name(Path(cfg.model_path).name)
models_ir = load_openvino_models(vino_dir, cfg)

if not models_ir:
    raise RuntimeError("No OpenVINO models found.")

print("Running OpenVINO inference...")
if len(dataset) > 0:
    row_ids, predictions = run_inference_openvino(dataset, models_ir, cfg, species_ids)
else:
    print("No test data available, generating empty submission.")
    row_ids = []
    predictions = []

# smoothing前の予測値を保存
submission_df = utils_lib.create_submission(row_ids, predictions, species_ids, cfg)
submission_path = os.path.join(cfg.OUTPUT_DIR, 'submission_before_smoothing.csv')
submission_df.to_csv(submission_path, index=False)

print(f"Submission saved to {submission_path}")


Generating dataset...


Parallel melspec gen:   0%|          | 0/4 [00:00<?, ?it/s]

Loading OpenVINO models...
✅ Loaded model: model_fold0.xml
🎉 Total 1 OpenVINO model(s) loaded from ../models/fold0_safezone1000_head_hoplength512_vino
Running OpenVINO inference...
Creating submission dataframe...
Submission saved to ../data/result/submission_before_smoothing.csv


In [43]:
def apply_power_to_low_ranked_cols(df: pd.DataFrame, top_k: int = 30, exponent: float = 2.0) -> pd.DataFrame:
    result = df.copy()
    class_cols = df.columns[1:]  # 1列目は row_id

    for idx, row in df.iterrows():
        scores = row[class_cols].values.astype(np.float32)  # 明示的に float32 に変換
        sorted_indices = np.argsort(scores)[::-1]
        top_indices = sorted_indices[:top_k]
        low_indices = sorted_indices[top_k:]

        adjusted_scores = scores.copy()
        adjusted_scores[low_indices] = adjusted_scores[low_indices] ** exponent

        # float32 に再キャストしてから代入
        result.iloc[idx, 1:] = adjusted_scores.astype(np.float32)

    return result

# Apply power transformation to low-ranked columns
submission_df = apply_power_to_low_ranked_cols(submission_df, top_k=30, exponent=2.5)

In [44]:
# smoothingして再度保存．
smoothed_df = smooth_submission_df(submission_df, cfg)
smoothed_submission_path = os.path.join(cfg.OUTPUT_DIR, 'submission.csv')
smoothed_df.to_csv(smoothed_submission_path, index=False)
print(f"Smoothed submission saved to {smoothed_submission_path}")

Smoothing submission predictions with global average blend...
Smoothed submission saved to ../data/result/submission.csv


In [45]:
# 提出用ファイルを読み込む
submission = pd.read_csv(os.path.join(cfg.OUTPUT_DIR, 'submission.csv'))
submission.head(12)

,row_id,1139490,1192948,1194042,126247,1346504,134933,135045,1462711,1462737,...,yebfly1,yebsee1,yecspi2,yectyr1,yehbla2,yehcar1,yelori1,yeofly1,yercac1,ywcpar
0,H02_20230502_080500_5,5.041683e-13,1.308637e-12,1.271597e-12,4.284660e-14,1.484300e-12,4.343004e-13,4.259835e-10,9.663776e-13,7.763028e-13,...,0.000637,0.000229,0.005839,0.000061,0.000537,9.589667e-10,0.001497,1.942161e-10,0.000580,2.213181e-12
1,H02_20230502_080500_10,2.604109e-13,6.485609e-13,6.268327e-13,7.134863e-14,7.272604e-13,4.109687e-13,5.293843e-10,7.146073e-13,3.521033e-13,...,0.001214,0.000297,0.010790,0.000061,0.000981,9.288372e-10,0.001497,2.951556e-10,0.001057,3.223371e-12
2,H02_20230502_080500_15,1.338535e-13,3.112935e-13,3.233372e-13,1.274386e-13,3.708904e-13,5.730911e-13,8.578308e-10,7.550168e-13,1.405212e-13,...,0.000797,0.000653,0.020549,0.000061,0.001870,5.118348e-10,0.001689,1.525890e-10,0.000580,5.263403e-12
3,H02_20230502_080500_20,6.824864e-14,1.592643e-13,1.097262e-13,8.477989e-14,1.678136e-13,6.205478e-13,4.484974e-10,4.415632e-13,3.723510e-14,...,0.000429,0.001118,0.011225,0.000061,0.000981,3.626132e-10,0.003957,8.191729e-11,0.000341,4.794670e-12
4,H02_20230502_080500_25,4.516744e-14,1.742342e-13,7.524047e-14,5.165890e-14,1.365022e-13,7.213094e-13,2.457174e-10,2.574573e-13,3.642117e-14,...,0.000114,0.002001,0.006128,0.000119,0.000537,1.404741e-10,0.006641,1.605683e-11,0.000103,5.616854e-12
5,H02_20230502_080500_30,3.151353e-14,1.869381e-13,4.935820e-14,2.446926e-14,1.283111e-13,5.906455e-13,5.024162e-11,1.042285e-13,3.380121e-14,...,0.000060,0.001843,0.001176,0.000332,0.000093,9.965330e-11,0.012680,1.547215e-11,0.000103,4.560684e-12
6,H02_20230502_080500_35,2.410147e-14,2.336386e-13,4.593991e-14,2.454651e-14,1.460003e-13,6.918877e-13,4.739432e-11,8.065793e-14,3.307295e-14,...,0.000060,0.002216,0.001032,0.000603,0.000093,7.280004e-11,0.010828,1.572899e-11,0.000103,5.235213e-12
7,H02_20230502_080500_40,3.447736e-14,4.766952e-13,4.682632e-14,3.667209e-14,1.849055e-13,9.433772e-13,4.741909e-11,1.398519e-13,5.085941e-14,...,0.000060,0.001079,0.001032,0.000794,0.000093,9.253739e-11,0.012202,1.689559e-11,0.000381,8.436455e-12
8,H02_20230502_080500_45,4.491383e-14,6.379433e-13,4.775808e-14,2.873565e-14,1.484524e-13,1.264631e-12,4.746555e-11,1.988380e-13,6.665462e-14,...,0.000060,0.000714,0.001032,0.000427,0.000093,1.257821e-10,0.008192,1.854707e-11,0.000660,8.458365e-12
9,H02_20230502_080500_50,6.425679e-14,9.766321e-13,4.966560e-14,3.939409e-14,1.497273e-13,2.012827e-12,4.752612e-11,3.068429e-13,9.790325e-14,...,0.000060,0.000593,0.001032,0.000285,0.000093,1.111053e-10,0.008762,1.918730e-11,0.001217,1.143250e-11


In [46]:
submission = pd.read_csv(os.path.join(cfg.OUTPUT_DIR, 'submission_before_smoothing.csv'))
submission.head(12)

,row_id,1139490,1192948,1194042,126247,1346504,134933,135045,1462711,1462737,...,yebfly1,yebsee1,yecspi2,yectyr1,yehbla2,yehcar1,yelori1,yeofly1,yercac1,ywcpar
0,H02_20230502_080500_5,1.473898e-05,2.144495e-05,2.144495e-05,2.406103e-06,0.000023,0.000011,1.584362e-04,1.777855e-05,1.777855e-05,...,0.000553,0.000043,1.088969e-04,0.000315,0.000043,2.611904e-04,0.000090,0.000096,0.000431,0.000007
1,H02_20230502_080500_10,5.422219e-06,6.144175e-06,5.422219e-06,1.653690e-06,0.000005,0.000002,7.967302e-05,7.411280e-06,1.994730e-06,...,0.003273,0.000038,4.878571e-04,0.000217,0.000180,3.150387e-04,0.000883,0.000231,0.002981,0.000019
2,H02_20230502_080500_15,5.093706e-06,7.811483e-07,9.516251e-06,1.012999e-05,0.000008,0.000013,3.569707e-04,1.892515e-05,6.893611e-07,...,0.000667,0.000315,6.008665e-02,0.000045,0.005555,3.120191e-05,0.000096,0.000020,0.000024,0.000038
3,H02_20230502_080500_20,6.144175e-06,5.771920e-06,4.495182e-06,5.093706e-06,0.000008,0.000013,6.605214e-05,1.012999e-05,3.089494e-06,...,0.000626,0.000856,1.810211e-03,0.000043,0.000570,1.911043e-04,0.000109,0.000038,0.000131,0.000018
4,H02_20230502_080500_25,2.726471e-06,7.411280e-06,1.003014e-06,2.902312e-06,0.000005,0.000016,8.398072e-06,3.726639e-06,2.902312e-06,...,0.000191,0.003594,3.763645e-05,0.000102,0.000096,3.288749e-06,0.002397,0.000005,0.000245,0.000040
5,H02_20230502_080500_30,3.927862e-07,2.902312e-06,2.238029e-07,1.003014e-06,0.000003,0.000002,3.059023e-07,6.083590e-07,4.737905e-07,...,0.000004,0.000503,7.338210e-07,0.000180,0.000010,8.851568e-07,0.025957,0.000005,0.000043,0.000011
6,H02_20230502_080500_35,1.003014e-06,6.962257e-06,6.893611e-07,1.653690e-06,0.000006,0.000017,1.067703e-06,1.994730e-06,7.811483e-07,...,0.000080,0.005060,6.540437e-06,0.000732,0.000004,1.287895e-06,0.002801,0.000023,0.000058,0.000023
7,H02_20230502_080500_40,1.873876e-06,1.078331e-05,5.715004e-07,5.422219e-06,0.000010,0.000011,8.851568e-07,3.089494e-06,2.726471e-06,...,0.000031,0.000710,7.411280e-06,0.001927,0.000029,5.771920e-06,0.025565,0.000014,0.000131,0.000045
8,H02_20230502_080500_45,2.123378e-06,8.398072e-06,4.450851e-07,6.893611e-07,0.000002,0.000008,5.422219e-06,3.966986e-06,2.406103e-06,...,0.000149,0.000755,5.829126e-05,0.000570,0.000066,1.233946e-04,0.000315,0.000033,0.000856,0.000020
9,H02_20230502_080500_50,6.962257e-06,2.282800e-05,2.726471e-06,4.222833e-06,0.000008,0.000031,1.147877e-05,1.384600e-05,8.398072e-06,...,0.000368,0.000417,4.006370e-05,0.000040,0.000038,9.028039e-05,0.000261,0.000043,0.003483,0.000058


In [47]:
print("✅ Shape:", submission.shape)
print("✅ Columns:", submission.columns.tolist())
print("✅ Dtypes:\n", submission.dtypes)
print("✅ Nulls:\n", submission.isna().sum().sum())

✅ Shape: (48, 207)
✅ Columns: ['row_id', '1139490', '1192948', '1194042', '126247', '1346504', '134933', '135045', '1462711', '1462737', '1564122', '21038', '21116', '21211', '22333', '22973', '22976', '24272', '24292', '24322', '41663', '41778', '41970', '42007', '42087', '42113', '46010', '47067', '476537', '476538', '48124', '50186', '517119', '523060', '528041', '52884', '548639', '555086', '555142', '566513', '64862', '65336', '65344', '65349', '65373', '65419', '65448', '65547', '65962', '66016', '66531', '66578', '66893', '67082', '67252', '714022', '715170', '787625', '81930', '868458', '963335', 'amakin1', 'amekes', 'ampkin1', 'anhing', 'babwar', 'bafibi1', 'banana', 'baymac', 'bbwduc', 'bicwre1', 'bkcdon', 'bkmtou1', 'blbgra1', 'blbwre1', 'blcant4', 'blchaw1', 'blcjay1', 'blctit1', 'blhpar1', 'blkvul', 'bobfly1', 'bobher1', 'brtpar1', 'bubcur1', 'bubwre1', 'bucmot3', 'bugtan', 'butsal1', 'cargra1', 'cattyr', 'chbant1', 'chfmac1', 'cinbec1', 'cocher1', 'cocwoo1', 'colara1', 'c